In [1]:
import os
import platform
import cv2 as cv
import json
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers, models, optimizers, backend
from sys import path
path.insert(0, '..')
import model
import config
import utill
from layer import conv, _make_divisible, conv_block, inverted_res_block

2022-03-09 06:17:38.153360: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-09 06:17:38.153410: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def MobileNetV2(inputs, num_classes, anchor_size):
    first_block_filters = _make_divisible(32, 8)
    x = conv_block(inputs=inputs, filters=first_block_filters,
                   kernel=(3, 3), strides=(2, 2), block_id=0)

    x = inverted_res_block(x, filters=16, strides=1, expansion=1, block_id=1)
    print(x.shape)
    x = inverted_res_block(x, filters=24, strides=2, expansion=6, block_id=2)
    print(x.shape)
    x = inverted_res_block(x, filters=24, strides=1, expansion=6, block_id=3)
    print(x.shape)
    x = inverted_res_block(x, filters=32, strides=2, expansion=6, block_id=4)
    print(x.shape)
    x = inverted_res_block(x, filters=32, strides=1, expansion=6, block_id=5)
    print(x.shape)
    x = inverted_res_block(x, filters=32, strides=1, expansion=6, block_id=6)
    print(x.shape)
    x = inverted_res_block(x, filters=64, strides=2, expansion=6, block_id=7)
    print(x.shape)
    x = inverted_res_block(x, filters=64, strides=1, expansion=6, block_id=8)
    print(x.shape)
    x = inverted_res_block(x, filters=64, strides=1, expansion=6, block_id=9)
    print(x.shape)
    x = inverted_res_block(x, filters=64, strides=1, expansion=6, block_id=10)
    print(x.shape)
    x = inverted_res_block(x, filters=96, strides=1, expansion=6, block_id=11)
    print(x.shape)
    x = inverted_res_block(x, filters=96, strides=1, expansion=6, block_id=12)
    print(x.shape)
    x = inverted_res_block(x, filters=96, strides=1, expansion=6, block_id=13)
    print(x.shape)
    x = inverted_res_block(x, filters=160, strides=2, expansion=6, block_id=14)
    print(x.shape)
    x = inverted_res_block(x, filters=160, strides=1, expansion=6, block_id=15)
    print(x.shape)
    x = inverted_res_block(x, filters=160, strides=1, expansion=6, block_id=16)
    print(x.shape)
    x = inverted_res_block(x, filters=320, strides=1, expansion=6, block_id=17)
    print(x.shape)
    x = conv_block(x, 1280, kernel=(1, 1), block_id=18)
    print(x.shape)
    outputs = conv(x, anchor_size * (num_classes + 5), 1, activation=None, batch_norm=False)

    return models.Model(inputs, outputs, name='mobilenetv2')
    

In [3]:
inputs = layers.Input((448, 448, 3))
mdl = MobileNetV2(inputs, 8, 7)

2022-03-09 06:17:45.339337: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-09 06:17:45.339379: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ozmonday): /proc/driver/nvidia/version does not exist
2022-03-09 06:17:45.339728: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 223, 223, 16)
(None, 111, 111, 24)
(None, 110, 110, 24)
(None, 55, 55, 32)
(None, 54, 54, 32)
(None, 53, 53, 32)
(None, 26, 26, 64)
(None, 25, 25, 64)
(None, 24, 24, 64)
(None, 23, 23, 64)
(None, 22, 22, 96)
(None, 21, 21, 96)
(None, 20, 20, 96)
(None, 10, 10, 160)
(None, 9, 9, 160)
(None, 8, 8, 160)
(None, 7, 7, 320)
(None, 7, 7, 1280)


In [27]:
mdl.summary()


Model: "mobilenetv2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 416, 416, 3)]     0         
_________________________________________________________________
conv_block_0 (Conv2D)        (None, 208, 208, 32)      864       
_________________________________________________________________
conv_block_0_bn (BatchNormal (None, 208, 208, 32)      128       
_________________________________________________________________
conv_block_0_relu (ReLU)     (None, 208, 208, 32)      0         
_________________________________________________________________
conv_block_1 (Conv2D)        (None, 208, 208, 32)      1024      
_________________________________________________________________
conv_block_1_bn (BatchNormal (None, 208, 208, 32)      128       
_________________________________________________________________
conv_block_1_relu (ReLU)     (None, 208, 208, 32)      

In [8]:
print(mdl.outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 15, 15, 91), dtype=tf.float32, name=None), name='conv2d/BiasAdd:0', description="created by layer 'conv2d'")
